In [128]:
from datetime import datetime, timedelta
from pulp import LpProblem, LpVariable, lpSum, LpBinary, LpMinimize, value, LpBinary, LpStatus
import json

# Define the problem
prob = LpProblem("nurse_scheduling_optimization", LpMinimize)

# Parameters
start_date = datetime(2024, 1, 1)
num_days = 31
dates = [(start_date + timedelta(days=i)) for i in range(num_days)]
shifts = ["day", "evening", "night"]
nurses = list(range(32))
positions = ["Nurse", "Head", "CNL"]
max_shifts_per_month = 25
max_shifts_per_day = 2

# Define shift demands as a dictionary
demand = {
    "Saturday": {"day": {"Nurse": 6, "CNL": 1}, "evening": {"Nurse": 5, "Head": 1}, "night": {"Nurse": 5, "Head": 1}},
    "Sunday": {"day": {"Nurse": 6, "CNL": 1}, "evening": {"Nurse": 5, "Head": 1}, "night": {"Nurse": 5, "Head": 1}},
    "Monday": {"day": {"Nurse": 6, "CNL": 1}, "evening": {"Nurse": 5, "Head": 1}, "night": {"Nurse": 5, "Head": 1}},
    "Tuesday": {"day": {"Nurse": 6, "CNL": 1}, "evening": {"Nurse": 5, "Head": 1}, "night": {"Nurse": 5, "Head": 1}},
    "Wednesday": {"day": {"Nurse": 6, "CNL": 1}, "evening": {"Nurse": 5, "Head": 1}, "night": {"Nurse": 5, "Head": 1}},
    "Thursday": {"day": {"Nurse": 6, "CNL": 1}, "evening": {"Nurse": 5, "Head": 1}, "night": {"Nurse": 5, "Head": 1}},
    "Friday": {"day": {"Nurse": 6, "Head": 1}, "evening": {"Nurse": 5, "Head": 1}, "night": {"Nurse": 5, "Head": 1}},
}

# Decision variables
assignments = LpVariable.dicts(
    "Assignments",
    ((n, s, d, p) for n in nurses for s in shifts for d in dates for p in positions),
    cat=LpBinary
)

# Objective function: Minimize total assignments
prob += lpSum(assignments)

# Constraints based on shift demand
for d in dates:
    weekday = d.strftime("%A")
    for s in shifts:
        for p in demand[weekday][s]:
            required_count = demand[weekday][s][p]
            prob += lpSum(assignments[n, s, d, p] for n in nurses) == required_count

# Max shifts per month constraint
for n in nurses:
    prob += lpSum(assignments[n, s, d, p] for s in shifts for d in dates for p in positions) <= max_shifts_per_month

# Max shifts per day constraint
for n in nurses:
    for d in dates:
        prob += lpSum(assignments[n, s, d, p] for s in shifts for p in positions) <= max_shifts_per_day

for n in nurses:
    for d in range(len(dates) - 1):  
        prob += (
            lpSum(assignments[n, s, dates[d + 1],p] for s in shifts) + assignments[n, "night", dates[d], p] <= 1 
        )
# Solve the problem
prob.solve()

# Output results

def format_output_as_json(assignments, shifts, dates):
    output = []
    for d in dates:
        weekday = d.strftime("%A")
        for s in shifts:
            assigned_nurses = []
            for n in nurses:
                for p in positions:
                    if assignments[n, s, d, p].varValue == 1:
                        assigned_nurses.append({
                            "id": n,
                            "role": p
                        })
            if assigned_nurses:
                head_nurse = next((n for n in assigned_nurses if n["role"] == "Head"), None)
                cnl_nurse = next((n for n in assigned_nurses if n["role"] == "CNL"), None)
                output.append({
                    "date": d.strftime("%Y-%m-%d"),
                    "weekday": weekday,
                    "shift": s,
                    "nurses_assigned": assigned_nurses,
                    "head_nurse": head_nurse if head_nurse else {"id": None, "role": "None"},
                    "cnl_nurse": cnl_nurse if cnl_nurse else {"id": None, "role": "None"} if s == "day" and weekday != "Friday" else {"id": None, "role": "None"}
                })
    return json.dumps(output, indent=4)

# Generate JSON output
json_output = format_output_as_json(assignments, shifts, dates)

# Print JSON output
print(json_output)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/miniconda3/envs/DS/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_z/mbltc11j1kxchc3w07g4ryr40000gn/T/9476e9f616d6438f9449bc7e45253b2f-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_z/mbltc11j1kxchc3w07g4ryr40000gn/T/9476e9f616d6438f9449bc7e45253b2f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2175 COLUMNS
At line 56608 RHS
At line 58779 BOUNDS
At line 67708 ENDATA
Problem MODEL has 2170 rows, 8928 columns and 27648 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 589 - 0.02 seconds
Cgl0004I processed model has 2170 rows, 5952 columns (5952 integer (5952 of which binary)) and 20864 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 589
Cbc

In [124]:
demand["Saturday"]["day"]["CNL"]

1

In [129]:
from pulp import LpStatus
print("Status:", LpStatus[prob.status])

Status: Optimal
